In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import types
import pandas_datareader as pdr
from BackTestAlgo import *
from Order import *
from DataQuery import *
from Record import *
from AlphaBeta import *
from Preprocessing import *

In [2]:
def my_init(context):
    context.symbols = ['MRNA', 'NVDA', 'AMZN', 'ARVL']
    # universe를 symbols로 정의(반드시 지정해야함)

    context.price = 'Adj Close'
    # price로 사용할 칼럼 인덱스 이름을 알려줘야함(open, high, close 등의 인덱스를 price로 활용 가능함)
    # 여기서 지정한 인덱스는 포트폴리오를 평가할 때 기본적으로 사용하는 지표임

    context.capital_base = 0
    # 투자원금을 설정함. 지정하지 않으면 기본값을 사용함

    context.i = 0

In [3]:
def handle_data(context, data):
    context.i += 1
    print('\n')
    print(data.current_time)

    order_schedule = pd.read_csv("C:/Users/ajcltm/Desktop/ordr_schedule.csv")
    order_schedule['date'] = pd.to_datetime(order_schedule['date'], format='%Y-%m-%d')
    order_schedule = order_schedule[['date', 'symbol', 'price', 'amounts']]

    for i in range(0, len(order_schedule)) :
        if data.current_time == order_schedule.loc[i, 'date']:
            symbol = order_schedule.loc[i, 'symbol']
            price = order_schedule.loc[i, 'price']
            amounts = order_schedule.loc[i, 'amounts']
            deposit(context, price*amounts)
            order(context, [symbol], [price], [amounts])

    mrna_price = data.current_data('MRNA', 'price')
    mrna_amounts = context.portfolio['stock']['MRNA']['amounts']
    nvda_price = data.current_data('NVDA', 'price')
    nvda_amounts = context.portfolio['stock']['NVDA']['amounts']
    amzn_price = data.current_data('AMZN', 'price')
    amzn_amounts = context.portfolio['stock']['AMZN']['amounts']
    arvl_price = data.current_data('ARVL', 'price')
    arvl_amounts = context.portfolio['stock']['ARVL']['amounts']

    bench_return = context.benchmark['benchmark_class'].benchmark_history(context, context.benchmark['benchmark_symbols'], 'return', 1).values[0]


    record(context, bench_return=bench_return, mrna_price=mrna_price, mrna_amounts=mrna_amounts, nvda_price=nvda_price, nvda_amounts=nvda_amounts, amzn_price=amzn_price, amzn_amounts = amzn_amounts, arvl_price=arvl_price, arvl_amounts = arvl_amounts)


In [4]:
s_date = datetime.datetime(2020, 9, 2)
e_date = datetime.datetime(2021, 6, 28)

mrna = pdr.DataReader('MRNA', 'yahoo', s_date, e_date)
nvda = pdr.DataReader('NVDA', 'yahoo', s_date, e_date)
amzn = pdr.DataReader('AMZN', 'yahoo', s_date, e_date)
arvl = pdr.DataReader('ARVL', 'yahoo', s_date, e_date)

mrna['date'] = mrna.index
mrna = mrna.reset_index(drop=True)
mrna['symbol'] = 'MRNA'
mrna = mrna[['date', 'symbol', 'Adj Close']]

nvda['date'] = nvda.index
nvda = nvda.reset_index(drop=True)
nvda['symbol'] = 'NVDA'
nvda = nvda[['date', 'symbol', 'Adj Close']]

amzn['date'] = amzn.index
amzn = amzn.reset_index(drop=True)
amzn['symbol'] = 'AMZN'
amzn = amzn[['date', 'symbol', 'Adj Close']]

arvl['date'] = arvl.index
arvl = arvl.reset_index(drop=True)
arvl['symbol'] = 'ARVL'
arvl = arvl[['date', 'symbol', 'Adj Close']]

data = pd.concat([mrna, nvda, amzn, arvl])

In [5]:
start_date = datetime.datetime(2020, 9, 2)
end_date = datetime.datetime(2021, 6, 28)
benchmark = pdr.DataReader('^GSPC', 'yahoo', start_date, end_date)
benchmark['date'] = benchmark.index
benchmark = benchmark.reset_index(drop=True)
benchmark['price'] = benchmark['Adj Close']
benchmark['benchmark'] = 'SP500'
benchmark = benchmark[['date', 'benchmark', 'price']]

checking_data_contidion(my_init, data, benchmark)


<Cheking Data Columns>
 - necesserly columns : ['date', 'symbol']
 - data columns : ['date', 'symbol', 'Adj Close']
 - data symbols not in context symbols : Nothing

<Cheking Symbols>
 - length of context symbols : 4
 - context symbols : ['MRNA', 'NVDA', 'AMZN', 'ARVL']
 - length of data symbols : 4
 - data symbols : ['MRNA', 'NVDA', 'AMZN', 'ARVL']
 - context symbols not in data symbols : Nothing
 - data symbols not in context symbols : Nothing

<Cheking Data imformation>
 - Please be sure "date" column should be the type of datetime
<class 'pandas.core.frame.DataFrame'>
Int64Index: 828 entries, 0 to 206
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   date       828 non-null    datetime64[ns]
 1   symbol     828 non-null    object        
 2   Adj Close  828 non-null    float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 25.9+ KB
 - None

<Checking date_univers>
 - date univer

In [6]:
tester = BackTester(initialize=my_init, tradingAlgo=handle_data)

result = tester.run(data, benchmark)


Hello backtesting
context.benchmarks symbols : SP500
context.benchmark_data : 
          date benchmark        price    return
0   2020-09-01     SP500  3526.649902       NaN
1   2020-09-02     SP500  3580.840088  0.015366
2   2020-09-03     SP500  3455.060059 -0.035126
3   2020-09-04     SP500  3426.959961 -0.008133
4   2020-09-08     SP500  3331.840088 -0.027756
..         ...       ...          ...       ...
202 2021-06-22     SP500  4246.439941  0.005124
203 2021-06-23     SP500  4241.839844 -0.001083
204 2021-06-24     SP500  4266.490234  0.005811
205 2021-06-25     SP500  4280.700195  0.003331
206 2021-06-28     SP500  4290.609863  0.002315

[207 rows x 4 columns]


2020-09-01 00:00:00
periods : 1
querydata : 
        date benchmark        price  return
0 2020-09-01     SP500  3526.649902     NaN
y : 
[[nan]]
y drop : 
[]
periods : 1
querydata : 
        date benchmark        price  return
0 2020-09-01     SP500  3526.649902     NaN
benchmark_history : 
0   NaN
Name: return, dty

C:\Users\ajcltm\PycharmProjects\BackTesting\BackTestAlgo.py:124: RuntimeWarning: invalid value encountered in double_scalars
  rate_of_return = (ending_portfolio_value - capital_base) / capital_base
C:\Users\ajcltm\PycharmProjects\BackTesting\BackTestAlgo.py:126: RuntimeWarning: invalid value encountered in double_scalars
  portfolio_return = (ending_portfolio_value - starting_portfolio_value) / starting_portfolio_value


y : 
[[        nan]
 [ 0.0802687 ]
 [-0.07534676]
 [-0.03106313]
 [-0.07126904]
 [ 0.06356537]
 [-0.02378934]]
y drop : 
[[ 0.0802687 ]
 [-0.07534676]
 [-0.03106313]
 [-0.07126904]
 [ 0.06356537]
 [-0.02378934]]
periods : 7
querydata : 
        date benchmark        price    return
0 2020-09-01     SP500  3526.649902       NaN
1 2020-09-02     SP500  3580.840088  0.015366
2 2020-09-03     SP500  3455.060059 -0.035126
3 2020-09-04     SP500  3426.959961 -0.008133
4 2020-09-08     SP500  3331.840088 -0.027756
5 2020-09-09     SP500  3398.959961  0.020145
6 2020-09-10     SP500  3339.189941 -0.017585
benchmark_history : 
0         NaN
1    0.015366
2   -0.035126
3   -0.008133
4   -0.027756
5    0.020145
6   -0.017585
Name: return, dtype: float64
x : 
[[ 0.01536591]
 [-0.03512584]
 [-0.00813303]
 [-0.02775634]
 [ 0.02014499]
 [-0.0175848 ]]
alpha : 
0.015705566832547486
beta_list : 
[2.861]


2020-09-11 00:00:00
periods : 1
querydata : 
        date benchmark        price    return
7 2020-


2020-09-22 00:00:00
periods : 1
querydata : 
         date benchmark        price    return
14 2020-09-22     SP500  3315.570068  0.010518
y : 
[[        nan]
 [ 0.0802687 ]
 [-0.07534676]
 [-0.03106313]
 [-0.07126904]
 [ 0.06356537]
 [-0.02378934]
 [-0.00382916]
 [ 0.06108328]
 [ 0.01739818]
 [-0.0231618 ]
 [-0.00617443]
 [-0.01104368]
 [ 0.0189645 ]
 [ 0.00584842]]
y drop : 
[[ 0.0802687 ]
 [-0.07534676]
 [-0.03106313]
 [-0.07126904]
 [ 0.06356537]
 [-0.02378934]
 [-0.00382916]
 [ 0.06108328]
 [ 0.01739818]
 [-0.0231618 ]
 [-0.00617443]
 [-0.01104368]
 [ 0.0189645 ]
 [ 0.00584842]]
periods : 15
querydata : 
         date benchmark        price    return
0  2020-09-01     SP500  3526.649902       NaN
1  2020-09-02     SP500  3580.840088  0.015366
2  2020-09-03     SP500  3455.060059 -0.035126
3  2020-09-04     SP500  3426.959961 -0.008133
4  2020-09-08     SP500  3331.840088 -0.027756
5  2020-09-09     SP500  3398.959961  0.020145
6  2020-09-10     SP500  3339.189941 -0.017585
7  202

querydata : 
         date benchmark        price    return
0  2020-09-01     SP500  3526.649902       NaN
1  2020-09-02     SP500  3580.840088  0.015366
2  2020-09-03     SP500  3455.060059 -0.035126
3  2020-09-04     SP500  3426.959961 -0.008133
4  2020-09-08     SP500  3331.840088 -0.027756
5  2020-09-09     SP500  3398.959961  0.020145
6  2020-09-10     SP500  3339.189941 -0.017585
7  2020-09-11     SP500  3340.969971  0.000533
8  2020-09-14     SP500  3383.540039  0.012742
9  2020-09-15     SP500  3401.199951  0.005219
10 2020-09-16     SP500  3385.489990 -0.004619
11 2020-09-17     SP500  3357.010010 -0.008412
12 2020-09-18     SP500  3319.469971 -0.011183
13 2020-09-21     SP500  3281.060059 -0.011571
14 2020-09-22     SP500  3315.570068  0.010518
15 2020-09-23     SP500  3236.919922 -0.023721
16 2020-09-24     SP500  3246.590088  0.002987
17 2020-09-25     SP500  3298.459961  0.015977
18 2020-09-28     SP500  3351.600098  0.016111
19 2020-09-29     SP500  3335.469971 -0.004813


...

y : 
[[        nan]
 [ 0.0802687 ]
 [-0.07534676]
 [-0.03106313]
 [-0.07126904]
 [ 0.06356537]
 [-0.02378934]
 [-0.00382916]
 [ 0.06108328]
 [ 0.01739818]
 [-0.0231618 ]
 [-0.00617443]
 [-0.01104368]
 [ 0.0189645 ]
 [ 0.00584842]
 [-0.03682763]
 [ 0.00805077]
 [-0.17525304]
 [ 0.01252526]
 [ 0.01463388]
 [ 0.02304201]
 [ 0.00620815]
 [-0.04056338]
 [ 0.05384105]
 [-0.01565266]
 [ 0.02494352]
 [-0.00463567]
 [ 0.01553825]
 [ 0.04198011]
 [ 0.00074458]
 [-0.01824365]
 [-0.0080261 ]
 [-0.01638179]
 [-0.02110278]
 [ 0.00622919]
 [-0.00951636]
 [-0.00649288]
 [ 0.01216953]
 [-0.01285812]
 [ 0.02263353]
 [-0.04926449]
 [ 0.02432811]
 [-0.04495205]
 [-0.00237418]
 [ 0.02475623]
 [ 0.06097477]
 [ 0.02589318]
 [ 0.01591927]
 [-0.05878824]
 [-0.05198763]
 [ 0.04397446]
 [-0.00165395]
 [-0.00489656]
 [ 0.01020679]
 [-0.00360981]
 [-0.0034988 ]
 [ 0.00197804]
 [-0.01816187]
 [ 0.00227803]
 [-0.00586389]
 [ 0.02142277]
 [ 0.00249222]
 [ 0.00290586]
 [ 0.00599667]
 [ 0.00485574]
 [-0.00851857]
 [ 0.

y : 
[[        nan]
 [ 0.0802687 ]
 [-0.07534676]
 [-0.03106313]
 [-0.07126904]
 [ 0.06356537]
 [-0.02378934]
 [-0.00382916]
 [ 0.06108328]
 [ 0.01739818]
 [-0.0231618 ]
 [-0.00617443]
 [-0.01104368]
 [ 0.0189645 ]
 [ 0.00584842]
 [-0.03682763]
 [ 0.00805077]
 [-0.17525304]
 [ 0.01252526]
 [ 0.01463388]
 [ 0.02304201]
 [ 0.00620815]
 [-0.04056338]
 [ 0.05384105]
 [-0.01565266]
 [ 0.02494352]
 [-0.00463567]
 [ 0.01553825]
 [ 0.04198011]
 [ 0.00074458]
 [-0.01824365]
 [-0.0080261 ]
 [-0.01638179]
 [-0.02110278]
 [ 0.00622919]
 [-0.00951636]
 [-0.00649288]
 [ 0.01216953]
 [-0.01285812]
 [ 0.02263353]
 [-0.04926449]
 [ 0.02432811]
 [-0.04495205]
 [-0.00237418]
 [ 0.02475623]
 [ 0.06097477]
 [ 0.02589318]
 [ 0.01591927]
 [-0.05878824]
 [-0.05198763]
 [ 0.04397446]
 [-0.00165395]
 [-0.00489656]
 [ 0.01020679]
 [-0.00360981]
 [-0.0034988 ]
 [ 0.00197804]
 [-0.01816187]
 [ 0.00227803]
 [-0.00586389]
 [ 0.02142277]
 [ 0.00249222]
 [ 0.00290586]
 [ 0.00599667]
 [ 0.00485574]
 [-0.00851857]
 [ 0.

periods : 1
querydata : 
          date benchmark        price    return
192 2021-06-08     SP500  4227.259766  0.000175
y : 
[[        nan]
 [ 0.0802687 ]
 [-0.07534676]
 [-0.03106313]
 [-0.07126904]
 [ 0.06356537]
 [-0.02378934]
 [-0.00382916]
 [ 0.06108328]
 [ 0.01739818]
 [-0.0231618 ]
 [-0.00617443]
 [-0.01104368]
 [ 0.0189645 ]
 [ 0.00584842]
 [-0.03682763]
 [ 0.00805077]
 [-0.17525304]
 [ 0.01252526]
 [ 0.01463388]
 [ 0.02304201]
 [ 0.00620815]
 [-0.04056338]
 [ 0.05384105]
 [-0.01565266]
 [ 0.02494352]
 [-0.00463567]
 [ 0.01553825]
 [ 0.04198011]
 [ 0.00074458]
 [-0.01824365]
 [-0.0080261 ]
 [-0.01638179]
 [-0.02110278]
 [ 0.00622919]
 [-0.00951636]
 [-0.00649288]
 [ 0.01216953]
 [-0.01285812]
 [ 0.02263353]
 [-0.04926449]
 [ 0.02432811]
 [-0.04495205]
 [-0.00237418]
 [ 0.02475623]
 [ 0.06097477]
 [ 0.02589318]
 [ 0.01591927]
 [-0.05878824]
 [-0.05198763]
 [ 0.04397446]
 [-0.00165395]
 [-0.00489656]
 [ 0.01020679]
 [-0.00360981]
 [-0.0034988 ]
 [ 0.00197804]
 [-0.01816187]
 [ 0

periods : 1
querydata : 
          date benchmark        price    return
197 2021-06-15     SP500  4246.589844 -0.002012
y : 
[[        nan]
 [ 0.0802687 ]
 [-0.07534676]
 [-0.03106313]
 [-0.07126904]
 [ 0.06356537]
 [-0.02378934]
 [-0.00382916]
 [ 0.06108328]
 [ 0.01739818]
 [-0.0231618 ]
 [-0.00617443]
 [-0.01104368]
 [ 0.0189645 ]
 [ 0.00584842]
 [-0.03682763]
 [ 0.00805077]
 [-0.17525304]
 [ 0.01252526]
 [ 0.01463388]
 [ 0.02304201]
 [ 0.00620815]
 [-0.04056338]
 [ 0.05384105]
 [-0.01565266]
 [ 0.02494352]
 [-0.00463567]
 [ 0.01553825]
 [ 0.04198011]
 [ 0.00074458]
 [-0.01824365]
 [-0.0080261 ]
 [-0.01638179]
 [-0.02110278]
 [ 0.00622919]
 [-0.00951636]
 [-0.00649288]
 [ 0.01216953]
 [-0.01285812]
 [ 0.02263353]
 [-0.04926449]
 [ 0.02432811]
 [-0.04495205]
 [-0.00237418]
 [ 0.02475623]
 [ 0.06097477]
 [ 0.02589318]
 [ 0.01591927]
 [-0.05878824]
 [-0.05198763]
 [ 0.04397446]
 [-0.00165395]
 [-0.00489656]
 [ 0.01020679]
 [-0.00360981]
 [-0.0034988 ]
 [ 0.00197804]
 [-0.01816187]
 [ 0


2021-06-22 00:00:00
periods : 1
querydata : 
          date benchmark        price    return
202 2021-06-22     SP500  4246.439941  0.005124
y : 
[[        nan]
 [ 0.0802687 ]
 [-0.07534676]
 [-0.03106313]
 [-0.07126904]
 [ 0.06356537]
 [-0.02378934]
 [-0.00382916]
 [ 0.06108328]
 [ 0.01739818]
 [-0.0231618 ]
 [-0.00617443]
 [-0.01104368]
 [ 0.0189645 ]
 [ 0.00584842]
 [-0.03682763]
 [ 0.00805077]
 [-0.17525304]
 [ 0.01252526]
 [ 0.01463388]
 [ 0.02304201]
 [ 0.00620815]
 [-0.04056338]
 [ 0.05384105]
 [-0.01565266]
 [ 0.02494352]
 [-0.00463567]
 [ 0.01553825]
 [ 0.04198011]
 [ 0.00074458]
 [-0.01824365]
 [-0.0080261 ]
 [-0.01638179]
 [-0.02110278]
 [ 0.00622919]
 [-0.00951636]
 [-0.00649288]
 [ 0.01216953]
 [-0.01285812]
 [ 0.02263353]
 [-0.04926449]
 [ 0.02432811]
 [-0.04495205]
 [-0.00237418]
 [ 0.02475623]
 [ 0.06097477]
 [ 0.02589318]
 [ 0.01591927]
 [-0.05878824]
 [-0.05198763]
 [ 0.04397446]
 [-0.00165395]
 [-0.00489656]
 [ 0.01020679]
 [-0.00360981]
 [-0.0034988 ]
 [ 0.00197804

In [8]:
pd.set_option('display.max.columns', 50)
pd.set_option('display.max_rows', 1000)
result

,date,total_profit,rate_of_return,starting_cash,ending_cash,starting_stock_value,ending_stock_value,starting_portfolio_value,ending_portfolio_value,portfolio_return,capital_base,alpha,beta_SP500,bench_return,mrna_price,mrna_amounts,nvda_price,nvda_amounts,amzn_price,amzn_amounts,arvl_price,arvl_amounts
0,2020-09-01,0.000000,NaN,0,0,0,0.000000,0,0.000000,NaN,0,NaN,NaN,NaN,63.320000,0,552.359436,0,3499.120117,0,9.970000,0
1,2020-09-02,104.442417,0.080269,1301.16,0.0,0.0,1405.602417,1301.16,1405.602417,0.080269,1301.16,0.080269,0.000,0.015366,64.720001,4,573.361206,2,3531.449951,0,10.025000,0
2,2020-09-03,-1.465176,-0.001126,0.0,0.0,1405.602417,1299.694824,1405.602417,1299.694824,-0.075347,1301.16,0.032911,3.082,-0.035126,64.839996,4,520.167419,2,3368.000000,0,10.010000,0
3,2020-09-04,-41.837765,-0.032154,0.0,0.0,1299.694824,1259.322235,1299.694824,1259.322235,-0.031063,1301.16,0.019612,3.047,-0.008133,62.599998,4,504.461121,2,3294.620117,0,10.010000,0
4,2020-09-08,-131.588452,-0.101132,0.0,0.0,1259.322235,1169.571548,1259.322235,1169.571548,-0.071269,1301.16,0.018826,3.104,-0.027756,54.340000,4,476.105774,2,3149.840088,0,10.010000,0
5,2020-09-09,-57.244198,-0.043995,0.0,0.0,1169.571548,1243.915802,1169.571548,1243.915802,0.063565,1301.16,0.013871,2.907,0.020145,56.900002,4,508.157898,2,3268.610107,0,10.010000,0
6,2020-09-10,-86.836132,-0.066737,0.0,0.0,1243.915802,1214.323868,1243.915802,1214.323868,-0.023789,1301.16,0.015706,2.861,-0.017585,57.560001,4,492.041931,2,3175.110107,0,10.050000,0
7,2020-09-11,-91.485974,-0.070311,0.0,0.0,1214.323868,1209.674026,1214.323868,1209.674026,-0.003829,1301.16,0.012213,2.796,0.000533,59.340000,4,486.157013,2,3116.219971,0,10.073000,0
8,2020-09-14,-17.595120,-0.013523,0.0,0.0,1209.674026,1283.564880,1209.674026,1283.564880,0.061083,1301.16,0.014259,2.875,0.012742,63.669998,4,514.442444,2,3102.969971,0,10.010000,0
9,2020-09-15,4.736576,0.003640,0.0,0.0,1283.56488,1305.896576,1283.56488,1305.896576,0.017398,1301.16,0.012807,2.840,0.005219,66.879997,4,519.188293,2,3156.129883,0,10.000000,0


In [12]:
date = [datetime.datetime(2020,1,i) for i in range(1,11)]
free_risk = pd.DataFrame(data={'date':date, 'return': [.1 for i in range(0, 10)]})
free_risk

,date,return
0,2020-01-01,0.1
1,2020-01-02,0.1
2,2020-01-03,0.1
3,2020-01-04,0.1
4,2020-01-05,0.1
5,2020-01-06,0.1
6,2020-01-07,0.1
7,2020-01-08,0.1
8,2020-01-09,0.1
9,2020-01-10,0.1


In [15]:
free_risk.iloc[:5+1]['return'].values.reshape(-1, 1)

array([[0.1],
       [0.1],
       [0.1],
       [0.1],
       [0.1],
       [0.1]])

In [1]:
pip install jupyterthemes

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\users\ajcltm\desktop\python\pythonproject\scripts\python.exe -m pip install --upgrade pip' command.


In [2]:
jt -l

NameError: name 'jt' is not defined